In [50]:
def commonly_used_mpin(mpin):
    # Check that MPIN is either 4 or 6 digits
    if len(mpin) not in [4, 6] or not mpin.isdigit():
        return False

    # Check for all same digits
    if len(set(mpin)) == 1:
        return True

    # Check if digits are in ascending order
    def is_ascending(mpin):
        for i in range(len(mpin) - 1):
            if int(mpin[i+1]) - int(mpin[i]) != 1:
                return False
        return True

    # Check if digits are in descending order 
    def is_descending(mpin):
        for i in range(len(mpin) - 1):
            if int(mpin[i]) - int(mpin[i+1]) != 1:
                return False
        return True

    # Check if MPIN is a palindrome
    def is_palindrome(mpin):
        return mpin == mpin[::-1]

    # Check for repeating patterns
    def has_pattern(mpin):
        length = len(mpin)
        for size in range(1, length // 2 + 1):
            if length % size == 0:
                part = mpin[:size]
                if part * (length // size) == mpin:
                    return True
        return False

    # Apply all checks
    if is_ascending(mpin) or is_descending(mpin):
        return True
    if is_palindrome(mpin):
        return True
    if has_pattern(mpin):
        return True

    return False


In [51]:

def mpin_demographic_check(mpin, date):
    
    if len(mpin) not in [4, 6] or not mpin.isdigit():
        return False

    day = date[:2]
    month = date[2:4]
    year = date[4:8]
    year_first = year[:2]
    year_last = year[-2:]

    parts = [day, month, year_first, year_last, year]

    # Direct full year or reversed year match
    if year == mpin or year[::-1] == mpin:
        return True


    for i in range(len(parts)):
        for j in range(len(parts)):
            for k in range(len(parts)):
                combs = set()
                if i != j and len(parts[i] + parts[j]) == len(mpin):
                    combs.add(parts[i] + parts[j])
                    combs.add(parts[j] + parts[i])
                if i != j and j != k and i != k and len(parts[i] + parts[j] + parts[k]) == len(mpin):
                    combs.add(parts[i] + parts[j] + parts[k])
                    combs.add(parts[k] + parts[j] + parts[i])
                if mpin in combs:
                    return True

    return False

def mpin_check_between_demographics(mpin, demographic1, demographic2):
    demographic1_division = [
        demographic1[:2], demographic1[2:4],
        demographic1[4:8], demographic1[4:6], demographic1[6:8]
    ]
    demographic2_division = [
        demographic2[:2], demographic2[2:4],
        demographic2[4:8], demographic2[4:6], demographic2[6:8]
    ]

    for p1 in demographic1_division:
        for p2 in demographic2_division:
            comb1 = p1 + p2
            comb2 = p2 + p1
            if len(comb1) in [4, 6] and (comb1 == mpin or comb2 == mpin):
                return True

    return False





In [52]:
def mpin_strength_check(mpin, dob_self=None, dob_spouse=None, anniversary=None):
   
    reasons = []
    
    # Validate MPIN format
    if len(mpin) not in [4, 6] or not mpin.isdigit():
        return "INVALID", ["INVALID_MPIN"]

    # Check if MPIN is commonly used
    if commonly_used_mpin(mpin):
        reasons.append("COMMONLY_USED")

    # Dictionary of provided demographic dates
    demographics = {
        "DEMOGRAPHIC_DOB_SELF": dob_self,
        "DEMOGRAPHIC_DOB_SPOUSE": dob_spouse,
        "DEMOGRAPHIC_ANNIVERSARY": anniversary
    }

    # Check if MPIN is directly or indirectly derived from any single demographic date
    for reason, date in demographics.items():
        if date and mpin_demographic_check(mpin, date):
            reasons.append(reason)

    found = False
    # Check if MPIN is a combination across different demographic dates
    dates = [d for d in demographics.values() if d]
    for i in range(len(dates)):
        for j in range(i + 1, len(dates)):
            if mpin_check_between_demographics(mpin, dates[i], dates[j]):
                reasons.append("DEMOGRAPHIC_COMBINATION")
                found = True
                break  
        if found:
            break
    strength = "WEAK" if reasons else "STRONG"
    return strength, reasons


In [ ]:
def date_formatter(date_str):
    date_str = date_str.strip()

    # if date is already without symbal then assuming ddmmyyyy format returning the date
    if date_str.isdigit() and len(date_str) == 8:
        return date_str

    
    for symbol in ['-', '/', '.']:
        if symbol in date_str:
            parts = date_str.split(symbol)
            if len(parts) == 3:
                p1, p2, p3 = parts
                if len(p3) == 4: 
                    if int(p1) > 12:  
                        return f"{p1.zfill(2)}{p2.zfill(2)}{p3}"
                    elif int(p2) > 12: 
                        return f"{p2.zfill(2)}{p1.zfill(2)}{p3}"
                    else:
                        # assuming dd mm yyyy -> dd mm yyyy
                        return f"{p1.zfill(2)}{p2.zfill(2)}{p3}"
                    
                elif len(p1) == 4:
                    if int(p2) > 12:  
                        return f"{p2.zfill(2)}{p3.zfill(2)}{p1}"
                    elif int(p3) > 12: 
                        return f"{p3.zfill(2)}{p2.zfill(2)}{p1}"
                    else:
                        # assuming yyyy mm dd -> dd mm yyyy
                        return f"{p3.zfill(2)}{p2.zfill(2)}{p1}"
                    
                else: 
                    if int(p1) > 12:  
                        return f"{p1.zfill(2)}{p3.zfill(2)}{p2}"
                    elif int(p3) > 12: 
                        return f"{p3.zfill(2)}{p1.zfill(2)}{p2}"
                    else:
                        # assuming dd yyyy mm -> dd mm yyyy
                        return f"{p1.zfill(2)}{p3.zfill(2)}{p2}"
                    
    # cases where we don't know the exact date format 
    # we assume ddmmyyyy format as in India usually this format is used.
    # We would need third party library or information from user or format hint to user
    # to get exact format type. 
    return ''.join(filter(str.isdigit, date_str))


In [54]:
def run_testcases(mpin, dob_self=None, dob_spouse=None, anniversary=None, expected_strength='', expected_reasons=None):
    formatted_dob_self = date_formatter(dob_self)
    # print(formatted_dob_self)
    formatted_dob_spouse = date_formatter(dob_spouse)
    formatted_anniversary =  date_formatter(anniversary)
    strength, reasons = mpin_strength_check(mpin, formatted_dob_self, formatted_dob_spouse, formatted_anniversary)

    print(f"Testing MPIN: {mpin} ({len(mpin)}-digit)")
    print(f"DOB Self      : {dob_self}")
    print(f"DOB Spouse    : {dob_spouse}")
    print(f"Anniversary   : {anniversary}")
    print(f"Expected Strength: {expected_strength} | Actual Strength: {strength}")
    print(f"Expected Reasons : {expected_reasons} | Actual Reasons : {reasons}")

    if strength == expected_strength and set(reasons) == set(expected_reasons):
        print("Test Passed!")
    else:
        print("Test Failed!")

    print("-" * 100)


### Different dateformat testcases

In [55]:
# 1. Date in MM/DD/YYYY format with slashes (should normalize correctly)
run_testcases("1225", "12/25/1990", "03/15/1991", "05/10/2022", "WEAK", ["DEMOGRAPHIC_DOB_SELF"])

# 2. Date in DD-MM-YYYY with hyphens and palindrome MPIN
run_testcases("1221", "21-12-1990", "11-11-1991", "10-10-2010", "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SELF"])

# 3. MM/DD/YYYY format with both MM and DD ≤ 12 (ambiguous)
run_testcases("1204", "12/04/1990", "11/11/1991", "10/10/2010", "WEAK", ["DEMOGRAPHIC_DOB_SELF"])

# 4. YYYY-MM-DD format (should fallback to DDMMYYYY)
run_testcases("1990", "1990-01-01", "02-02-1991", "03-03-1992", "WEAK", [ "DEMOGRAPHIC_DOB_SELF", "DEMOGRAPHIC_COMBINATION"])

# 5. Dot separator and strong MPIN unrelated to date
run_testcases("763914", "03.04.1995", "07.08.1994", "11.11.1993", "STRONG", [])

# 6. Derived from spouse date with ambiguity in MM/DD and DD/MM
run_testcases("0312", "01-01-1990", "03-12-1991", "03-03-1992", "WEAK", ["DEMOGRAPHIC_DOB_SPOUSE", "DEMOGRAPHIC_COMBINATION"])

# 7. MPIN made from combo of DOB and Anniversary
run_testcases("0110", "01-01-1990", "05-05-1991", "10-10-2020", "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_COMBINATION"])

# 8. All symbols removed, assume Indian format fallback
run_testcases("0101", "01011990", "02021991", "03031992", "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SELF"])

# 9. Reverse anniversary year
run_testcases("0202", "01-01-2000", "02-02-2002", "03-03-2020", "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SPOUSE",  "DEMOGRAPHIC_ANNIVERSARY"])

# 10. Random but valid MPIN that shouldn’t match anything
run_testcases("918273", "01-01-1980", "05-07-1983", "11-12-1986", "STRONG", [])


Testing MPIN: 1225 (4-digit)
DOB Self      : 12/25/1990
DOB Spouse    : 03/15/1991
Anniversary   : 05/10/2022
Expected Strength: WEAK | Actual Strength: WEAK
Expected Reasons : ['DEMOGRAPHIC_DOB_SELF'] | Actual Reasons : ['DEMOGRAPHIC_DOB_SELF']
Test Passed!
----------------------------------------------------------------------------------------------------
Testing MPIN: 1221 (4-digit)
DOB Self      : 21-12-1990
DOB Spouse    : 11-11-1991
Anniversary   : 10-10-2010
Expected Strength: WEAK | Actual Strength: WEAK
Expected Reasons : ['COMMONLY_USED', 'DEMOGRAPHIC_DOB_SELF'] | Actual Reasons : ['COMMONLY_USED', 'DEMOGRAPHIC_DOB_SELF']
Test Passed!
----------------------------------------------------------------------------------------------------
Testing MPIN: 1204 (4-digit)
DOB Self      : 12/04/1990
DOB Spouse    : 11/11/1991
Anniversary   : 10/10/2010
Expected Strength: WEAK | Actual Strength: WEAK
Expected Reasons : ['DEMOGRAPHIC_DOB_SELF'] | Actual Reasons : ['DEMOGRAPHIC_DOB_SELF']


###  Testcases

In [56]:
# 1. All same digits (4-digit)
run_testcases("1111", "01011990", "02021991", "03031992", "WEAK", ["COMMONLY_USED"])

# 2. All same digits (6-digit)
run_testcases("111111", "01011990", "02021991", "03031992", "WEAK", ["COMMONLY_USED"])

# 3. Strong MPIN (4-digit)
run_testcases("7485", "01012000", "02021991", "03032023", "STRONG", [])

# 4. Strong MPIN (6-digit)
run_testcases("748593", "01012000", "02021991", "03032023", "STRONG", [])

# 5. Ascending (4-digit)
run_testcases("1234", "01012000", "02022021", "03032022", "WEAK", ["COMMONLY_USED"])

# 6. Ascending (6-digit)
run_testcases("123456", "01012000", "02022021", "03032022", "WEAK", ["COMMONLY_USED"])

# 7. DOB derived (4-digit)
run_testcases("0101", "01012000", "02021991", "03031992", "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SELF"])

# 8. DOB derived (6-digit)
run_testcases("010100", "01012000", "02021991", "03031992", "WEAK", ["DEMOGRAPHIC_DOB_SELF"])

# 9. Repeating pattern (4-digit)
run_testcases("1212", "12121990", "02022021", "03032022", "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SELF"])

# 10. Reverse year & demographic combo
run_testcases("199191", "01012000", "02021991", "03031992", "WEAK", ["DEMOGRAPHIC_DOB_SPOUSE"])

# 11. Descending (4-digit)
run_testcases("4321", "01012000", "02021991", "03032023", "WEAK", ["COMMONLY_USED"])

# 12. Descending (6-digit)
run_testcases("654321", "01012000", "02021991", "03032023", "WEAK", ["COMMONLY_USED"])

# 13. Random strong (4-digit)
run_testcases("8491", "01012000", "02021991", "03032023", "STRONG", [])

# 14. Random strong (6-digit)
run_testcases("849136", "01012000", "02021991", "03032023", "STRONG", [])

# 15. DOB Spouse match (4-digit)
run_testcases("0202", "01012000", "02021991", "03032023", "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SPOUSE"])

# 16. DOB Spouse match (6-digit)
run_testcases("020291", "01012000", "02021991", "03032023", "WEAK", ["DEMOGRAPHIC_DOB_SPOUSE"])

# 17. Palindrome + demographic (4-digit)
run_testcases("0220", "01012000", "02202020", "03032023", "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SPOUSE","DEMOGRAPHIC_COMBINATION"])

# 18. Repeating chunk pattern (6-digit)
run_testcases("123123", "01011990", "02021991", "03031992", "WEAK", ["COMMONLY_USED"])

# 19. Full reversed year match (6-digit)
run_testcases("199120", "01011991", "20021991", "03031992", "WEAK", ["DEMOGRAPHIC_DOB_SPOUSE","DEMOGRAPHIC_COMBINATION"])

# 20. Palindrome (6-digit)
run_testcases("122221", "01012000", "02022022", "03032023", "WEAK", ["COMMONLY_USED"])

# 21. Partial year + month combo (DOB & Anniversary)
run_testcases("200203", "01012000", "02021991", "03032002", "WEAK", ["DEMOGRAPHIC_ANNIVERSARY"])

# 22. Month + Year (reversed) from spouse DOB
run_testcases("912002", "01012000", "20021991", "03032023", "WEAK", ["DEMOGRAPHIC_DOB_SPOUSE"])

# 23. Strong MPIN with leading zeros
run_testcases("041385", "01012000", "02021991", "03032023", "STRONG", [])

# 24. Pattern: Repeated halves
run_testcases("484848", "01012000", "02021991", "03032023", "WEAK", ["COMMONLY_USED"])

# 25. Valid but derived from full reversed anniversary year
run_testcases("320203", "01012000", "02021991", "03032023", "STRONG", [])


Testing MPIN: 1111 (4-digit)
DOB Self      : 01011990
DOB Spouse    : 02021991
Anniversary   : 03031992
Expected Strength: WEAK | Actual Strength: WEAK
Expected Reasons : ['COMMONLY_USED'] | Actual Reasons : ['COMMONLY_USED']
Test Passed!
----------------------------------------------------------------------------------------------------
Testing MPIN: 111111 (6-digit)
DOB Self      : 01011990
DOB Spouse    : 02021991
Anniversary   : 03031992
Expected Strength: WEAK | Actual Strength: WEAK
Expected Reasons : ['COMMONLY_USED'] | Actual Reasons : ['COMMONLY_USED']
Test Passed!
----------------------------------------------------------------------------------------------------
Testing MPIN: 7485 (4-digit)
DOB Self      : 01012000
DOB Spouse    : 02021991
Anniversary   : 03032023
Expected Strength: STRONG | Actual Strength: STRONG
Expected Reasons : [] | Actual Reasons : []
Test Passed!
----------------------------------------------------------------------------------------------------
Test